<a href="https://colab.research.google.com/github/Hongzf/OLID/blob/main/ML_Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install wordsegment
!pip install transformers
!pip install trainer
!pip install attention

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython.display import HTML
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import os
import emoji
from wordsegment import load, segment
import pickle
import torch
from torch.utils.data import Dataset

import os,re
from bs4 import BeautifulSoup
from gensim import models

from transformers import BertTokenizer, RobertaTokenizer, get_cosine_schedule_with_warmup
from trainer import Trainer 
from torch import nn
from transformers import BertModel, BertForSequenceClassification, RobertaForSequenceClassification, RobertaModel
from attention import Attention
load()

In [ ]:
#optional Google drive integration
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OLID_PATH = './drive/MyDrive/OLID'
SAVE_PATH = '.drive/MyDrive/OLID/save'
LABEL_DICT = {
    'a': {'OFF': 0, 'NOT': 1},
    'b': {'TIN': 0, 'UNT': 1, 'NULL': 2},
    'c': {'IND': 0, 'GRP': 1, 'OTH': 2, 'NULL': 3}
}

TRAIN_PATH = os.path.join(OLID_PATH, 'olid-training-v1.0.tsv')

In [ ]:
task = 'c'
# model_name = 'roberta'
# model_size = 'base'
truncate = 512
epochs = 10
lr = 0.0001
wd = 0.0
bs = 50 #72
patience = 5

In [ ]:
def emoji2word(sents):
    return [emoji.demojize(sent) for sent in sents]

def remove_useless_punctuation(sents):
    for i, sent in enumerate(sents):
        sent = sent.replace(':', ' ')
        sent = sent.replace('_', ' ')
        sent = sent.replace('...', ' ')
        sents[i] = sent
    return sents

def remove_replicates(sents):
    # if there are multiple `@USER` tokens in a tweet, replace it with `@USERS`
    # because some tweets contain so many `@USER` which may cause redundant
    for i, sent in enumerate(sents):
        if sent.find('@USER') != sent.rfind('@USER'):
            sents[i] = sent.replace('@USER', '')
            sents[i] = '@USERS ' + sents[i]
    return sents

def replace_rare_words(sents):
    rare_words = {
        'URL': 'http'
    }
    for i, sent in enumerate(sents):
        for w in rare_words.keys():
            sents[i] = sent.replace(w, rare_words[w])
    return sents

def segment_hashtag(sents):
    # E.g. '#LunaticLeft' => 'lunatic left'
    for i, sent in enumerate(sents):
        sent_tokens = sent.split(' ')
        for j, t in enumerate(sent_tokens):
            if t.find('#') == 0:
                sent_tokens[j] = ' '.join(segment(t))
        sents[i] = ' '.join(sent_tokens)
    return sents


def process_tweets(tweets):
    tweets = emoji2word(tweets)
    tweets = replace_rare_words(tweets)
    tweets = remove_replicates(tweets)
    tweets = segment_hashtag(tweets)
    tweets = remove_useless_punctuation(tweets)
    tweets = np.array(tweets)
    return tweets

In [ ]:
from pandas.io.formats.info import DataFrameInfo
df = pd.read_csv(TRAIN_PATH, sep='\t', keep_default_na=False)

if task == 'a':
        df = df
elif task == 'b':
        df=df[~df['subtask_b'].isin(['NULL'])]
elif task == 'c':
        df=df[~df['subtask_c'].isin(['NULL'])]


ids = np.array(df['id'].values)
tweets = np.array(df['tweet'].values)

print(tweets[1])
# Process tweets
df['tweet'] = process_tweets(tweets)

tweets_2 = df['tweet']
print(tweets_2[1])

label_a = np.array(df['subtask_a'].values)
label_b = df['subtask_b'].values
label_c = np.array(df['subtask_c'].values)
nums = len(df)

if task == 'a':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_b','subtask_c'],axis = 1)
elif task == 'b':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_a','subtask_c'],axis = 1)
        df_train=df_train[~df_train['subtask_b'].isin(['NULL'])]
elif task == 'c':
        df_train = df.copy()
        df_train = df_train.drop(['subtask_a','subtask_b'],axis = 1)
        df_train=df_train[~df_train['subtask_c'].isin(['NULL'])]



# read_test_file(task, tokenizer, truncate=512):
df1 = pd.read_csv(os.path.join(OLID_PATH, 'testset-level' + task + '.tsv'), sep='\t')
df2 = pd.read_csv(os.path.join(OLID_PATH, 'labels-level' + task + '.csv'), sep=',')
ids = np.array(df1['id'].values)
# tweets = np.array(df1['tweet'].values)
# labels = np.array(df2['label'].values)
# nums = len(df1)
df_test = pd.merge(df1,df2,on='id')
# Process tweets
tweets_test = df_test['tweet']
df_test['tweet'] = process_tweets(tweets_test)  

In [ ]:
x_train = tweets_2
x_val = tweets_test
Y_train_name = 'subtask_'+ task
y_train = df_train[Y_train_name]
y_val = df_test['label']

In [ ]:
x_total = pd.concat([x_train,x_val])
print(x_total)

In [ ]:
df_test['label'].isnull().any

In [ ]:
y_train.isnull().any

In [ ]:
x_val.isnull().any

In [ ]:
y_val.isnull().any

In [ ]:
y_train

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


num_features = 300   
min_word_count = 1    
num_workers = 4        
context = 10          
model_ = 0             

model_name = '{}features_{}minwords_{}context.model'.format(num_features, min_word_count, context)

model_name_2 = 'fasttext.model'

print('Training model...')
model_2 = models.FastText(x_total, size=num_features, window=context, min_count=min_word_count,\
                        sg = model_, min_n = 3 , max_n = 2, alpha=0.025, min_alpha=0.00025)
# , min_n = 2 , max_n = 3
#Save
# model_2.save(os.path.join('..', 'models', model_name_2))
# model_2.wv.save_word2vec_format(os.path.join('..','models','fasttext.txt'),binary = False)

if task == 'a':
        model_2.save('./drive/MyDrive/OLID/save/ML_embedding/fasttext_model')
elif task == 'b':
        model_2.save('./drive/MyDrive/OLID/save/ML_embedding/b_fasttext_model')
elif task == 'c':
        model_2.save('./drive/MyDrive/OLID/save/ML_embedding/c_fasttext_model')

model_2.save('./drive/MyDrive/OLID/save/ML_embedding/fasttext_model')
# model_2.wv.save_word2vec_format(('./drive/MyDrive/OLID/save/ML_embedding/fasttext.txt'),binary = False)


In [ ]:
model_name_fast = 'fasttext.model'

if task == 'a':
        fasttext_embedding = models.FastText.load(('./drive/MyDrive/OLID/save/ML_embedding/fasttext_model'))
elif task == 'b':
        fasttext_embedding = models.FastText.load(('./drive/MyDrive/OLID/save/ML_embedding/b_fasttext_model'))
elif task == 'c':
        fasttext_embedding = models.FastText.load(('./drive/MyDrive/OLID/save/ML_embedding/c_fasttext_model'))
        
# model_file =('./drive/MyDrive/OLID/save/ML_embedding/fasttext_model')
# fasttext_embedding = model.load_fasttext_format（model_file，encoding ='utf8'，full_model = True ）

In [ ]:
stop_words="english"
def stopwords_clean(text):
    text = [w for w in text if w not in stop_words]
    return text


def to_review_vector(review,model='fasttext'):
    words = process_tweets(review)
    if model == 'word2vec':
        array = np.asarray([word2vec_embedding[w] for w in words if w in word2vec_embedding],dtype='float32')
    elif model == 'glove':
        array = np.asarray([glove_embedding[w] for w in words if w in glove_embedding],dtype='float32')
    elif model == 'fasttext':
        array = np.asarray([fasttext_embedding[w] for w in words if w in fasttext_embedding],dtype='float32')
    else:
        raise ValueError('INPUT：word2vec、glove or fasttext')
    return array.mean(axis=0)
    

In [ ]:
"""sample"""
train_data_fasttext = [to_review_vector(text,'fasttext') for text in df['tweet']]

In [ ]:
test_data_fasttext = [to_review_vector(text,'fasttext') for text in df_test['tweet']]

In [ ]:
for item in train_data_fasttext:
    if np.isnan(item).any():
        print('yes')

In [ ]:
np.isnan(train_data_fasttext).any()

False

In [ ]:
np.isnan(test_data_fasttext).any()

False

Linear SVM


In [ ]:
from sklearn.svm import LinearSVC
# load the machine model form sklearn
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, naive_bayes, svm

# Define the parameters to tune
parameter_grid = { 
    'C': [1, 3],
    'gamma': [1, 'auto', 'scale']
}
# setup the SVM model
# SVM_model = GridSearchCV(svm.SVC(kernel='linear'), parameter_grid, cv=5, n_jobs=-1)
# SVM_model = LinearSVC(C=1, loss="hinge")
SVM_model = svm.SVC(kernel="linear", C=1)
# fit the model with training dataset
SVM_model.fit(train_data_fasttext, y_train)

SVC(C=1, kernel='linear')

In [ ]:
# predict the result labels on validation dataset
predictions_SVM = SVM_model.predict(test_data_fasttext)

In [ ]:
def model_eval(prediction):

    print("Confusion_Matrix：\n")
    print(metrics.confusion_matrix(y_val, prediction))

    print("Results：\n")
    print(metrics.classification_report(y_val, prediction))


In [ ]:
model_eval(predictions_SVM)

Random Forest

In [ ]:
RF = RandomForestClassifier()
RF.fit(train_data_fasttext, y_train)
# show model

RandomForestClassifier()

In [ ]:
predictions_RF = RF.predict(test_data_fasttext)

In [ ]:
model_eval(predictions_RF)

Logistic Regression

In [ ]:
LR = LogisticRegression()
LR.fit(train_data_fasttext, y_train)
# show model


In [ ]:
predictions_LR = LR.predict(test_data_fasttext)

In [ ]:
model_eval(predictions_LR)

In [ ]:
# on test dataset
cm = metrics.confusion_matrix(y_val, predictions_LR)

print('Confusion matrix：', cm, sep='\n')

TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

# print(FP, FN, TP, TN)

# accuracy: (TP + TN) / (TP + TN + FP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy: {:.3}%".format(accuracy*100))

# sensitivity(recall rate): TP / (TP + FN)
sensitivity = TP / (TP + FN)
print("Sensitivity: {:.3}%".format(sensitivity*100))

# specificity: TN / (FP + TN)
specificity = TN / (FP + TN)
print("Specificity: {:.3}%".format(specificity*100))

# precision: TP / (TP + FP)
precision = TP / (TP + FP)
print("Precision: {:.3}%".format(precision*100))

# F1 score: 2 * (precision * sensitivity) / (precision + sensitivity)
print("F1 score: {:.3}%".format(2 * (precision * sensitivity) / (precision + sensitivity)*100))

# classification report
r = metrics.classification_report(y_val, predictions_LR)
print('Classification report：', r, sep='\n')

In [ ]:
test_data_fasttext

In [ ]:
train_data_fasttext